In [2]:
import numpy as np
import matplotlib.pyplot as plt
import astroML.time_series
import os
%matplotlib inline

##Generalized Periodogram: 
* Theory/math detailed in [Zechmeister & Kürster, 2009](https://arxiv.org/abs/0901.2573)
* Code detailed [here](http://www.astroml.org/book_figures/chapter10/fig_LS_sg_comparison.html)

In [3]:
def lomb_scargle(time,flux,fluxerr,interval):
    '''Creates generalized periodogram using Zechmeister & Kürster, 2009 formalism,
    implemented in astroML package
    NOTE:sig is significance levels; 0.1=90%, 0.01=99%'''
    secondsElapsed=[]
    for i in range(0,len(time)):
        secondsElapsed.append((time[i]-time[0])*24*60*60+interval/2.0)
    omega=np.logspace(np.log10(2.*np.pi/(secondsElapsed[-1])),np.log10(2*np.pi/(2.*interval)),10000)
    lsg,sig=astroML.time_series.lomb_scargle(secondsElapsed,flux,fluxerr,omega,\
                                             generalized=True,significance=[0.1,0.01])
    return(lsg,sig,omega)

In [ ]:
#load lightcurves
path_dir='/path/to/files/'

#data file of mjd, flux, and flux error columns
data=np.loadtxt(path_dir+'lightcurve_file.txt')

mjd=data[:,0]
flux=data[:,1]
fluxerr=data[:,2]
time_bin=2.# in seconds

lsg,sig,omega=lomb_scargle(mjd,flux,fluxerr,time_bin)

In [ ]:
#log scale plot
fig=plt.figure()
ax1=fig.add_subplot(111)
ax1.plot(omega/(2*np.pi),lsg)
plt.xlim(min(omega)/(2.*np.pi),max(omega)/(2.*np.pi))
plt.xscale("log")
plt.yscale("log")
ax1.plot([omega[0]/(2*np.pi), omega[-1]/(2*np.pi)], [sig[0], sig[0]], '--', c='green',lw=2)
ax1.plot([omega[0]/(2*np.pi), omega[-1]/(2*np.pi)], [sig[1], sig[1]], '--', c='red',lw=2)
plt.xlabel('Frequency, $\\nu$ (Hz)',size=16)
plt.ylabel('Power',size=16)

In [ ]:
#linear scale plot
fig=plt.figure()
ax1=fig.add_subplot(111)
ax1.plot(omega/(2*np.pi),(lsg),marker='',linestyle='-')
ax1.plot([omega[0]/(2*np.pi), omega[-1]/(2*np.pi)], [sig[0], sig[0]], '--', c='green',lw=2)
ax1.plot([omega[0]/(2*np.pi), omega[-1]/(2*np.pi)], [sig[1], sig[1]], '--', c='red',lw=2)
plt.xscale("log")
plt.xlim(min(omega)/(2.*np.pi),max(omega)/(2.*np.pi))
plt.xlabel('Frequency, $\\nu$ (Hz)',size=16)
plt.ylabel('Power',size=16)